### 📌 **(ETL fase 01) - Extração Dados:**

In [206]:
import pandas as pd
import json

url = r'data\dadosAPI\TelecomX_Data.json'
df = pd.read_json(url)

In [207]:
df.head(3)

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


### 🔧 **(ETL fase 02) - Análise do Dataframe original + normalização:**

O dataframe provém de um <strong>arquivo.json</strong>, por isso a seguir, farei uso do método normalize de maneira
a deixar o documento mais adequado para manipulação.

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


In [209]:
df.dtypes

customerID    object
Churn         object
customer      object
phone         object
internet      object
account       object
dtype: object

In [210]:
colunas_df = list(df.columns)
colunas_df

['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']

In [211]:
pd.json_normalize(df['customer']).head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure
0,Female,0,Yes,Yes,9
1,Male,0,No,No,9


In [212]:
pd.json_normalize(df['phone']).head(2)

,PhoneService,MultipleLines
0,Yes,No
1,Yes,Yes


In [213]:
pd.json_normalize(df['internet']).head(2)

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,DSL,No,Yes,No,Yes,Yes,No
1,DSL,No,No,No,No,No,Yes


In [214]:
pd.json_normalize(df['account']).head(2)

,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,One year,Yes,Mailed check,65.6,593.3
1,Month-to-month,No,Mailed check,59.9,542.4


In [215]:
# Juntando subdataframes ao df original:
# exclui todas as colunas com objetos literais JS, e
# concatenei os dfs normalizados acima:

df = pd.concat([
    df.drop(columns=['customer','phone','internet','account']),
    pd.json_normalize(df['customer'], sep='_'),
    pd.json_normalize(df['phone'], sep='_'),
    pd.json_normalize(df['internet'], sep='_'),
    pd.json_normalize(df['account'], sep='_')
    ],
    axis=1)

In [216]:
df.head(3)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


In [217]:
df.set_index('customerID', inplace=True)

In [218]:
df.head(3)

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
customerID,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


In [219]:
# Analizando novas colunas existentes:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7267 entries, 0002-ORFBO to 9995-HOTOH
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7267 non-null   object 
 1   gender            7267 non-null   object 
 2   SeniorCitizen     7267 non-null   int64  
 3   Partner           7267 non-null   object 
 4   Dependents        7267 non-null   object 
 5   tenure            7267 non-null   int64  
 6   PhoneService      7267 non-null   object 
 7   MultipleLines     7267 non-null   object 
 8   InternetService   7267 non-null   object 
 9   OnlineSecurity    7267 non-null   object 
 10  OnlineBackup      7267 non-null   object 
 11  DeviceProtection  7267 non-null   object 
 12  TechSupport       7267 non-null   object 
 13  StreamingTV       7267 non-null   object 
 14  StreamingMovies   7267 non-null   object 
 15  Contract          7267 non-null   object 
 16  PaperlessBilling  7267 non-null 

In [220]:
df.dtypes

Churn                object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
Charges_Monthly     float64
Charges_Total        object
dtype: object

In [221]:
colunas_df = list(df.columns)
colunas_df

['Churn',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Charges_Monthly',
 'Charges_Total']

### 🔧 **(ETL fase 03) - Verificando Inconsistências nos Dados:**

In [222]:
def getValoresCelulas(df, nmColOrigem):
    vlsExistentes = {
        'nm_coluna': nmColOrigem,
        'valores_vistos': \
        list(pd.unique(df[nmColOrigem]))
    }
    return vlsExistentes

In [223]:
getValoresCelulas(df, 'Churn') # Valores preenchidos na coluna "Evasão/Churn".

{'nm_coluna': 'Churn', 'valores_vistos': ['No', 'Yes', '']}

In [224]:
for nmColuna in df.columns:
    print(getValoresCelulas(df, nmColuna))

{'nm_coluna': 'Churn', 'valores_vistos': ['No', 'Yes', '']}
{'nm_coluna': 'gender', 'valores_vistos': ['Female', 'Male']}
{'nm_coluna': 'SeniorCitizen', 'valores_vistos': [0, 1]}
{'nm_coluna': 'Partner', 'valores_vistos': ['Yes', 'No']}
{'nm_coluna': 'Dependents', 'valores_vistos': ['Yes', 'No']}
{'nm_coluna': 'tenure', 'valores_vistos': [9, 4, 13, 3, 71, 63, 7, 65, 54, 72, 5, 56, 34, 1, 45, 50, 23, 55, 26, 69, 11, 37, 49, 66, 67, 20, 43, 59, 12, 27, 2, 25, 29, 14, 35, 64, 39, 40, 6, 30, 70, 57, 58, 16, 32, 33, 10, 21, 61, 15, 44, 22, 24, 19, 47, 62, 46, 52, 8, 60, 48, 28, 41, 53, 68, 51, 31, 36, 17, 18, 38, 42, 0]}
{'nm_coluna': 'PhoneService', 'valores_vistos': ['Yes', 'No']}
{'nm_coluna': 'MultipleLines', 'valores_vistos': ['No', 'Yes', 'No phone service']}
{'nm_coluna': 'InternetService', 'valores_vistos': ['DSL', 'Fiber optic', 'No']}
{'nm_coluna': 'OnlineSecurity', 'valores_vistos': ['No', 'Yes', 'No internet service']}
{'nm_coluna': 'OnlineBackup', 'valores_vistos': ['Yes', 'No'

In [225]:
df['Charges_Monthly'].isna()

customerID
0002-ORFBO    False
0003-MKNFE    False
0004-TLHLJ    False
0011-IGKFF    False
0013-EXCHZ    False
              ...  
9987-LUTYD    False
9992-RRAMN    False
9992-UJOEL    False
9993-LHIEB    False
9995-HOTOH    False
Name: Charges_Monthly, Length: 7267, dtype: bool

In [226]:
df.columns.str.lower()

Index(['churn', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
       'charges_monthly', 'charges_total'],
      dtype='object')

In [227]:
df.columns.str.lower().str.replace(' ','_')

Index(['churn', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
       'charges_monthly', 'charges_total'],
      dtype='object')

In [228]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [229]:
df.head(3)

,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total
customerID,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


In [230]:
df.churn.unique()

array(['No', 'Yes', ''], dtype=object)

In [231]:
df['churn'].unique()

array(['No', 'Yes', ''], dtype=object)

### 🔧 **(ETL fase 04) - Tratando as Inconsistências nos Dados:**

Primeiro antes de criar a coluna "Contas Diárias" pedida no Challenge, excluirei alguns valores inválidos dentro do dataframe.
- Antes de converter valores aos tipos *np.float64*, *string*, *etc*; irei verificar valores inadequados, como strings vazias ou NaN.
- Feito isso, faço a conversão dos tipos.

In [238]:
# Testes iniciais:

a = df['churn'].isin(['Yes']).sum()
b = df['churn'].isin(['No']).sum()
(a + b) == len(df['churn'])

False

In [239]:
a + b

7043

In [240]:
selecao_churn1 = df['churn'] == 'Yes'
selecao_churn2 = df['churn'] == 'No'
len(df[selecao_churn1]) + \
len(df[selecao_churn2]) 

7043

In [243]:
c = df['churn'].isin(['Yes','No']).sum()
c

7043

In [244]:
c < len(df)

True

In [245]:
df['churn'].value_counts(dropna=False)

churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

In [247]:
# Série booleana com indices do DF que estão adequados
# ao padrão "Yes"/"No":
df['churn'].isin(['Yes','No'])

customerID
0002-ORFBO    True
0003-MKNFE    True
0004-TLHLJ    True
0011-IGKFF    True
0013-EXCHZ    True
              ... 
9987-LUTYD    True
9992-RRAMN    True
9992-UJOEL    True
9993-LHIEB    True
9995-HOTOH    True
Name: churn, Length: 7267, dtype: bool

In [249]:
# Indices os quais RESPEITAM o padrão acima:
df[df['churn'].isin(['Yes','No'])].head(3) 

,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total
customerID,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


In [250]:
# Indices os quais NÃO respeitam:
df[~df['churn'].isin(['Yes','No'])].head(3)

,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total
customerID,,,,,,,,,,,,,,,,,,,,
0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,Yes,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
0120-YZLQA,,Male,0,No,No,71,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,Yes,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2


In [252]:
len(df[~df['churn'].isin(['Yes','No'])]) # Número deles

224

In [257]:
df = df[df['churn'].isin(['Yes','No'])] # Retirando todos os índices indesejados do Dataframe

In [261]:
df['churn'].value_counts()

churn
No     5174
Yes    1869
Name: count, dtype: int64

In [263]:
df['churn'] = df['churn'].astype(str)

In [265]:
df = df[~df['charges_total'].isin(['',' '])] # Removendo valores vazios dos registros de cobrança.

In [278]:
import numpy as np

df.loc[:,'charges_total'] = df['charges_total'].astype(np.float64)

In [279]:
df.charges_total.head(3)

customerID
0002-ORFBO    593.30
0003-MKNFE    542.40
0004-TLHLJ    280.85
Name: charges_total, dtype: float64

In [281]:
df['charges_total'].head(3)

customerID
0002-ORFBO    593.30
0003-MKNFE    542.40
0004-TLHLJ    280.85
Name: charges_total, dtype: float64

In [282]:
df.charges_total.sum()

16056168.7

In [283]:
df.charges_monthly.sum()

455661.0

In [285]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0002-ORFBO to 9995-HOTOH
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   churn             7032 non-null   object 
 1   gender            7032 non-null   object 
 2   seniorcitizen     7032 non-null   int64  
 3   partner           7032 non-null   object 
 4   dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   phoneservice      7032 non-null   object 
 7   multiplelines     7032 non-null   object 
 8   internetservice   7032 non-null   object 
 9   onlinesecurity    7032 non-null   object 
 10  onlinebackup      7032 non-null   object 
 11  deviceprotection  7032 non-null   object 
 12  techsupport       7032 non-null   object 
 13  streamingtv       7032 non-null   object 
 14  streamingmovies   7032 non-null   object 
 15  contract          7032 non-null   object 
 16  paperlessbilling  7032 non-null 

In [296]:
contas_diarias_calc = (df['charges_monthly']/30)
contas_diarias_calc

customerID
0002-ORFBO    2.186667
0003-MKNFE    1.996667
0004-TLHLJ    2.463333
0011-IGKFF    3.266667
0013-EXCHZ    2.796667
                ...   
9987-LUTYD    1.838333
9992-RRAMN    2.836667
9992-UJOEL    1.676667
9993-LHIEB    2.261667
9995-HOTOH    1.966667
Name: charges_monthly, Length: 7032, dtype: float64

In [297]:
df['contas_diarias'] = contas_diarias_calc

In [298]:
df.columns

Index(['churn', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
       'charges_monthly', 'charges_total', 'contas_diarias'],
      dtype='object')

In [299]:
df.head(1)

,churn,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,charges_monthly,charges_total,contas_diarias
customerID,,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3,2.186667


### **📊 (ETL fase 05) - Carga e Análise (L - Load & Analysis)**

In [305]:
df.describe()

,seniorcitizen,tenure,charges_monthly,charges_total,contas_diarias
count,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.162400,32.421786,64.798208,2283.300441,2.159940
std,0.368844,24.545260,30.085974,2266.771362,1.002866
min,0.000000,1.000000,18.250000,18.800000,0.608333
25%,0.000000,9.000000,35.587500,401.450000,1.186250
50%,0.000000,29.000000,70.350000,1397.475000,2.345000
75%,0.000000,55.000000,89.862500,3794.737500,2.995417
max,1.000000,72.000000,118.750000,8684.800000,3.958333


In [308]:
df[['charges_monthly','charges_total','contas_diarias']].describe()

,charges_monthly,charges_total,contas_diarias
count,7032.000000,7032.000000,7032.000000
mean,64.798208,2283.300441,2.159940
std,30.085974,2266.771362,1.002866
min,18.250000,18.800000,0.608333
25%,35.587500,401.450000,1.186250
50%,70.350000,1397.475000,2.345000
75%,89.862500,3794.737500,2.995417
max,118.750000,8684.800000,3.958333


In [309]:
df.columns

Index(['churn', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
       'charges_monthly', 'charges_total', 'contas_diarias'],
      dtype='object')

In [ ]:
df[['']] # ... pergar o desvio padrão de month to manth, two years, one year